In [11]:
import dotenv

dotenv.load_dotenv()

True

In [12]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")


In [13]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
Analyze the every following code.
After anylyzing try to find some vulnerabilities in the code.
Here is a list of i wanna find: 
    - Reentrency Attack: Reentrancy is a vulnerability that allows an attacker to re-enter a function multiple times before the first function call is finished. so whenever the contract makes external call to other addresses, this is a possibility for reentrancy attack.This can lead to unexpected behavior, including reordering of transactions, and can be used to drain funds from a contract. 
    - Replay signatures attacks: The original account will sign a message then the delivery account will send the message to a smart contract, that way it is the delivery account that pays for the transaction fees and not the original account.
If you find anything, please provide a list of them in JSON format. Escape quotes in the JSON with a backslash. 
The JSON should contain the following fields:
- "line": The line number where the issue occurs.
- "token": The specific code token that is problematic.
- "problem": A short name for the issue.
- "severity": The severity of the issue (low, high).
- "explanation": A brief explanation of why this is a security risk.
- "migration": A brief explanation of how to fix this issue.
Code:
{code}
"""
)

In [14]:
import os

files = os.listdir('snippets')
code_snippets = []
for file in files:
    with open(f'snippets/{file}', 'r') as f:
        code_snippets.append(f.read())
print(code_snippets)

['// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBank{\n    mapping(address => uint) public credit;\n    \n    function deposit() public payable {\n        credit[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint amount) public{\n        if (credit[msg.sender]>= amount) {\n            require(msg.sender.call.value(amount)());\n            credit[msg.sender]-=amount;\n        }\n    }\n}', '// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBank is ReentrancyGuard {\n    mapping(address => uint) public credit;\n\n    function deposit() public payable {\n        credit[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint amount) public nonReentrant {\n        require(credit[msg.sender] >= amount, "Insufficient balance");\n        \n        credit[msg.sender] -= amount; // Effect: Update state before transferring funds\n        \n        (bool success,) = payable(msg.sender).call{value: amount}(""); // Interaction: T

In [15]:
from langchain.chains import LLMChain
chain = chain = prompt | model

In [16]:
code = code_snippets[2]

In [17]:
result = chain.invoke({"code": code})
result

AIMessage(content='Analyzing the given Solidity code for potential vulnerabilities, I have identified the following issues:\n\n```json\n[\n    {\n        "line": 17,\n        "token": "payable(msg.sender).transfer(amount)",\n        "problem": "Reentrancy Attack",\n        "severity": "high",\n        "explanation": "The contract transfers Ether to an external address before it updates the balance. This allows a malicious contract to re-enter the withdraw function before the state updates, potentially draining funds.",\n        "migration": "Use a \'checks-effects-interactions\' pattern by first updating the state (e.g., balance) and then performing the external call. Consider using `call` for transferring Ether with a gas limit and preventing reentrancy."\n    },\n    {\n        "line": 14,\n        "token": "bytes32 messageHash = keccak256(abi.encodePacked(msg.sender, amount, nonce))",\n        "problem": "Replay Signature Attack",\n        "severity": "high",\n        "explanation":

In [20]:
result_text = result.content.strip('`json')
print(code)
print(result_text)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract PiggyBridge {
    mapping(address => uint256) public balances;
    mapping(bytes32 => bool) public executedTxs;

    function deposit() external payable {
        balances[msg.sender] += msg.value;
    }

    function withdraw(uint256 amount, uint256 nonce, bytes memory signature) external nonReentrant {
        require(balances[msg.sender] >= amount, "Insufficient balance");

        bytes32 messageHash = keccak256(abi.encodePacked(msg.sender, amount, nonce));
        require(!executedTxs[messageHash], "Already executed");

        address signer = recoverSigner(messageHash, signature);
        require(signer == msg.sender, "Invalid signature");

        executedTxs[messageHash] = true;
        balances[msg.sender] -= amount;
        payable(msg.sender).transfer(amount);
    }

    function recoverSigner(bytes32 hash, bytes memory signature) public pure returns (address) {
        bytes32 ethSignedHash = keccak256(abi.e

In [19]:
import json
(result_text[result_text.find("```json") + len("```json"):result_text.find("```", result_text.find("```json") + len("```json"))])
json.loads(result_text[result_text.find("```json") + len("```json"):result_text.find("```", result_text.find("```json") + len("```json"))])

[{'line': 17,
  'token': 'payable(msg.sender).transfer(amount)',
  'problem': 'Reentrancy Attack',
  'severity': 'high',
  'explanation': 'The contract transfers Ether to an external address before it updates the balance. This allows a malicious contract to re-enter the withdraw function before the state updates, potentially draining funds.',
  'migration': "Use a 'checks-effects-interactions' pattern by first updating the state (e.g., balance) and then performing the external call. Consider using `call` for transferring Ether with a gas limit and preventing reentrancy."},
 {'line': 14,
  'token': 'bytes32 messageHash = keccak256(abi.encodePacked(msg.sender, amount, nonce))',
  'problem': 'Replay Signature Attack',
  'severity': 'high',
  'explanation': 'If the nonce is reused or not unique for every transaction, an attacker could replay the signed message on a different transaction, leading to unauthorized withdrawals.',
  'migration': "Ensure that the nonce is unique for each withdra